**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [18]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
import skvideo.io
#skvideo.setFFmpegPath("C:\\ffmpeg\\bin")
import skvideo
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D,Flatten, Activation, Dropout, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

In [2]:
skvideo.getFFmpegPath()

'C:\\ffmpeg\\bin'

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [6]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [7]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=False):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Epilson is used to make a balance between exploration and exploitation. The function act will use epsilon to select a random next action to force the algorithm to explore rather than just exploiting to find the optimal policies.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [8]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2,:]= -1
        self.position[:,:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[self.position < 0] = 0
        self.board[self.x,self.y] = 0
        
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [9]:
# parameters
size = 13
T= 200
temperature=0.3
epochs_train= 300 # set small when debugging
epochs_test= 50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))


__Question 2__ Explain the use of the arrays ```position``` and ```board```.


The array position will be used to inform the mouse about the possible positions(0) and the impossible ones (-1). The array board outlines where the cheese and poison are. Since the mouse sees $5^2$ steps ahead, the two arrays will display the cells accordling to the mouse's position. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [10]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [11]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score / e))
    print('Final score: {}'.format(score / epochs))

In [70]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=0.3)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 7.5/15.0. Average score (-7.5)
Win/lose count 6.0/13.0. Average score (-7.25)
Win/lose count 7.0/17.0. Average score (-8.166666666666666)
Win/lose count 12.0/16.0. Average score (-7.125)
Win/lose count 10.0/10.0. Average score (-5.7)
Win/lose count 10.0/14.0. Average score (-5.416666666666667)
Win/lose count 9.5/10.0. Average score (-4.714285714285714)
Win/lose count 9.5/13.0. Average score (-4.5625)
Win/lose count 4.5/9.0. Average score (-4.555555555555555)
Win/lose count 7.5/15.0. Average score (-4.85)
Win/lose count 7.5/11.0. Average score (-4.7272727272727275)
Win/lose count 8.0/13.0. Average score (-4.75)
Win/lose count 10.0/14.0. Average score (-4.6923076923076925)
Win/lose count 8.5/17.0. Average score (-4.964285714285714)
Win/lose count 9.5/11.0. Average score (-4.733333333333333)
Win/lose count 10.0/15.0. Average score (-4.75)
Win/lose count 2.0/7.0. Average score (-4.764705882352941)
Win/lose count 12.0/18.0. Average score (-4.833333333333333)
Win/lose count 10

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




#### Answer Q5

##### 1- Prove
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

We know that   $r(s,a)=E_{(s',a')\sim p(.|s,a)}(r(s,a))$  
and  $Q^{\pi}=E_{p^{\pi}}(\sum_{t\leq T} \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a)=r(s,a)+\gamma E_{p^{\pi}}(\sum_{t\leq T} \gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a)$  

##### By developing the first part 
We have $r(s,a)=E(R(s_{0}=s,a_{0}=a))$. As $(s_{1},a_{1})\sim p(.|s_{0},a_{0})$, so $(s_{1},a_{1})$ is $(s_{0},a_{0})$.
Then, we have $E(R(s_{0},a_{0}))=E(E(R(s_{0},a_{0}))|s_{1},a_{1})$

##### By developing the second part

Assume $X=\sum_{t\leq T} \gamma^{t}r(s_{t+1},a_{t+1}$. $X$ is $(s_{1},a_{1})$, then  $$E_{p^{\pi}}(X|s_{0},a_{0})=E_{p^{\pi}}(E_{p^{\pi}}(X|s_{1},a_{1})|s_{0},a_{0})=E_{(s_{1},a_{1})\sim{p^{\pi}}(.|s_{0},a_{0})}(E_{p^{\pi}}(X|s_{1},a_{1}))$$



For $T=\infty$, so we have $E_{p^{\pi}}(X|s_{1}=s',a_{1}=a')=Q^{\pi}(s',a')$, we will then have:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}




#### 2- Prove 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Let $\pi = (a_{0},\pi')$ then Q* or optimal Q is: 
\begin{align*}
Q^{\star}(s,a) &= \max_{\pi} Q^{\pi}(s,a) \\
&= r(s,a)  + \gamma\max_{a_{0},\pi'} \mathbb{E}_{(s',a')\sim (\mathbb{P}(.|s,a),\pi(.|s'))}(Q^{\pi}(s',a')) \\
&= r(s,a)  + \gamma\max_{a_{0}} \mathbb{E}_{s'\sim \mathbb{P}(.|s,a)}(Q^{\star}(s',a_{0})) \\
&= r(s,a)  + \gamma\mathbb{E}_{s'\sim \mathbb{P}(.|s,a)}(\max_{a_{0}} Q^{\star}(s',a_{0})) \\
\end{align*}



##### 3-Prove

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

The model becomes deterministic since we have Q*
so the objective $\mathcal{L}$ is plausible now:   $\mathcal{L}(\theta) = || r(s,a) + \gamma \max_{a'} Q(s',a',\theta) - Q(s,a,\theta)||^{2}$

For $\pi$ deterministic policy, we have $Q^{*}=Q^{\pi} \iff \pi=\pi^{*}$, with $\pi^{*}$.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [12]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
#         pass
        self.memory.append(m)
        self.memory = self.memory[:self.max_memory]

    def random_access(self):
#         pass
        rn = np.random.randint(0, len(self.memory))
        return self.memory[rn]

***
The pipeline we will use for training is given below:

In [13]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [35]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state # each state is 5x5x2

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # we only use V functions?
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            if game_over_:
                Qf = np.zeros(4)
            else:
                Qf = self.model.predict(n_s_[None])[0]
            #print(Qf)
            target_q[i][a_] = r_ + self.discount * np.max(Qf) # size 4 ??
            #print(np.max(Qf))
                
        ######## FILL IN target q and input states
        # HINT: Clip the target to avoid exploding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -2, 2)

        #print(input_states.shape, target_q.shape)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape = (5,5,self.n_state)))
        model.add(Dense(32,activation  = 'relu'))
        model.add(Dense(4,activation = 'linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [23]:
epochs_train= 50
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=200, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train40.mp4'))

Epoch 000/050 | Loss 0.0006 | Win/lose count 2.0/2.0 (0.0)
Epoch 001/050 | Loss 0.0035 | Win/lose count 0/1.0 (-1.0)
Epoch 002/050 | Loss 0.0028 | Win/lose count 0/1.0 (-1.0)
Epoch 003/050 | Loss 0.0003 | Win/lose count 0/1.0 (-1.0)
Epoch 004/050 | Loss 0.0001 | Win/lose count 3.0/4.0 (-1.0)
Epoch 005/050 | Loss 0.0015 | Win/lose count 0.5/0 (0.5)
Epoch 006/050 | Loss 0.0018 | Win/lose count 2.0/4.0 (-2.0)
Epoch 007/050 | Loss 0.0007 | Win/lose count 1.0/1.0 (0.0)
Epoch 008/050 | Loss 0.0017 | Win/lose count 2.0/3.0 (-1.0)
Epoch 009/050 | Loss 0.0017 | Win/lose count 0.5/1.0 (-0.5)
Epoch 010/050 | Loss 0.0009 | Win/lose count 1.5/1.0 (0.5)
Epoch 011/050 | Loss 0.0006 | Win/lose count 1.0/1.0 (0.0)
Epoch 012/050 | Loss 0.0003 | Win/lose count 1.0/0 (1.0)
Epoch 013/050 | Loss 0.0013 | Win/lose count 0/0 (0)
Epoch 014/050 | Loss 0.0009 | Win/lose count 1.0/0 (1.0)
Epoch 015/050 | Loss 0.0006 | Win/lose count 0.5/1.0 (-0.5)
Epoch 016/050 | Loss 0.0000 | Win/lose count 0/2.0 (-2.0)
Epoch 01

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [36]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32,kernel_size = (2,2), activation = 'relu',input_shape = (5,5,self.n_state)))
        model.add(Conv2D(32,kernel_size = (2,2), activation = 'relu'))
        model.add(Dropout(0.1))
        model.add(Flatten())
        model.add(Dense(32,activation = 'relu'))
        model.add(Dense(4,activation = 'linear'))
    
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [26]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=200, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train40.mp4'))

Epoch 000/050 | Loss 0.0096 | Win/lose count 3.0/2.0 (1.0)
Epoch 001/050 | Loss 0.0004 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/050 | Loss 0.0006 | Win/lose count 0/0 (0)
Epoch 003/050 | Loss 0.0005 | Win/lose count 0.5/0 (0.5)
Epoch 004/050 | Loss 0.0019 | Win/lose count 1.0/3.0 (-2.0)
Epoch 005/050 | Loss 0.0029 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/050 | Loss 0.0002 | Win/lose count 0/2.0 (-2.0)
Epoch 007/050 | Loss 0.0015 | Win/lose count 1.0/0 (1.0)
Epoch 008/050 | Loss 0.0034 | Win/lose count 0.5/0 (0.5)
Epoch 009/050 | Loss 0.0021 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/050 | Loss 0.0004 | Win/lose count 1.0/1.0 (0.0)
Epoch 011/050 | Loss 0.0005 | Win/lose count 2.5/0 (2.5)
Epoch 012/050 | Loss 0.0040 | Win/lose count 3.5/1.0 (2.5)
Epoch 013/050 | Loss 0.0018 | Win/lose count 5.0/1.0 (4.0)
Epoch 014/050 | Loss 0.0042 | Win/lose count 0.5/3.0 (-2.5)
Epoch 015/050 | Loss 0.0002 | Win/lose count 2.5/0 (2.5)
Epoch 016/050 | Loss 0.0008 | Win/lose count 2.0/3.0 (-1.0)
Epoch 017/0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [42]:
env = Environment(grid_size=size, max_time=T,temperature=0.4)
agent_cnn = DQN_CNN(size, lr=.09, epsilon = 0.1, memory_size=150, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.11, epsilon = 0.1, memory_size=150, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 2.0/0. Average score (1.0)
Win/lose count 0.5/0. Average score (0.8333333333333334)
Win/lose count 0/3.0. Average score (-0.125)
Win/lose count 0.5/2.0. Average score (-0.4)
Win/lose count 2.0/4.0. Average score (-0.6666666666666666)
Win/lose count 3.0/3.0. Average score (-0.5714285714285714)
Win/lose count 1.0/1.0. Average score (-0.5)
Win/lose count 0.5/1.0. Average score (-0.5)
Win/lose count 1.0/2.0. Average score (-0.55)
Win/lose count 2.0/1.0. Average score (-0.4090909090909091)
Win/lose count 1.0/0. Average score (-0.2916666666666667)
Win/lose count 1.0/1.0. Average score (-0.2692307692307692)
Win/lose count 1.5/3.0. Average score (-0.35714285714285715)
Win/lose count 3.5/1.0. Average score (-0.16666666666666666)
Win/lose count 4.0/0. Average score (0.09375)
Win/lose count 1.0/1.0. Average score (0.08823529411764706)
Win/lose count 4.0/5.0. Average score (0.027777777777777776)
Win/lose count 3.0/2.0. Average 

In [43]:
HTML(display_videos('cnn_test10.mp4'))

In [45]:
HTML(display_videos('fc_test10.mp4'))

It was noticed that the CNN outperforms the FC. Another thing noticed is that both algorithms get stuck in an area in the map (a local optima) which is due to a lack of exploration. 

By increasing the temperature, we immediately notice a higher average reward but the local optima problem remains present. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [46]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    score = 0
    loss = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
    
    def act(self, action,train=False):
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    def reset(self):

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [51]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.1)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0023 | Win/lose count 2.0/19.1 (-17.1)
Epoch 001/050 | Loss 0.0021 | Win/lose count 2.5/19.900000000000016 (-17.400000000000016)
Epoch 002/050 | Loss 0.0019 | Win/lose count 1.5/20.30000000000002 (-18.80000000000002)
Epoch 003/050 | Loss 0.0022 | Win/lose count 2.5/16.79999999999997 (-14.299999999999969)
Epoch 004/050 | Loss 0.0054 | Win/lose count 2.0/15.79999999999996 (-13.79999999999996)
Epoch 005/050 | Loss 0.0039 | Win/lose count 2.0/19.50000000000001 (-17.50000000000001)
Epoch 006/050 | Loss 0.0058 | Win/lose count 2.5/18.299999999999983 (-15.799999999999983)
Epoch 007/050 | Loss 0.0024 | Win/lose count 4.5/19.400000000000016 (-14.900000000000016)
Epoch 008/050 | Loss 0.0060 | Win/lose count 4.5/16.999999999999975 (-12.499999999999975)
Epoch 009/050 | Loss 0.0040 | Win/lose count 1.5/18.799999999999997 (-17.299999999999997)
Epoch 010/050 | Loss 0.0115 | Win/lose count 3.0/16.39999999999997 (-13.39999999999997)
Epoch 011/050 | Loss 0.0059 | Win/lose count 1.

In [52]:
HTML(display_videos('cnn_train_explore40.mp4'))

In [53]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore40.mp4'))

Win/lose count 0.5/0. Average score (0.5)
Win/lose count 1.0/1.0. Average score (0.25)
Win/lose count 4.0/0. Average score (1.5)
Win/lose count 3.5/0. Average score (2.0)
Win/lose count 0/0. Average score (1.6)
Win/lose count 1.0/0. Average score (1.5)
Win/lose count 2.5/1.0. Average score (1.5)
Win/lose count 0.5/0. Average score (1.375)
Win/lose count 0/1.0. Average score (1.1111111111111112)
Win/lose count 2.5/0. Average score (1.25)
Win/lose count 0.5/0. Average score (1.1818181818181819)
Win/lose count 0/0. Average score (1.0833333333333333)
Win/lose count 2.0/0. Average score (1.1538461538461537)
Win/lose count 0.5/0. Average score (1.1071428571428572)
Win/lose count 0/1.0. Average score (0.9666666666666667)
Win/lose count 0/0. Average score (0.90625)
Win/lose count 1.0/1.0. Average score (0.8529411764705882)
Win/lose count 1.5/0. Average score (0.8888888888888888)
Win/lose count 4.0/1.0. Average score (1.0)
Win/lose count 1.0/1.0. Average score (0.95)
Win/lose count 2.0/1.0. Ave

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***